In [2]:
import pandas as pd
data = pd.read_csv("spam.csv")
data.head()
data.groupby('Category').describe()
data['verdict']=data['Category'].apply(lambda x: 1 if x=='spam' else 0)
data.head()

,Category,Message,verdict
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(data.Message,data.verdict)
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:2]
model = MultinomialNB()
model.fit(X_train_count,y_train)
emails = [
    'Frii diskount here, bi fast',
    'click here to receive a free reward',
    'your car warranty is expired',
    'Do not fall for this trick!',
    'this is how much Dave saved with this simple trick',
    'you will not believe how simple this is!',
    'get your once a year discount today!',
    'receive your 30% discount here from your next product',
    'get your once a year discount today! Do not miss out on this HUGE oppurtunity!',
]

In [37]:
emails_count = v.transform(emails)
X_test_count = v.transform(X_test)
model.predict(emails_count), model.score(X_test_count, y_test)

(array([0, 1, 0, 0, 0, 0, 0, 1, 0], dtype=int64), 0.9842067480258435)

In [38]:
data2 = pd.read_parquet('spam2.parquet', engine='fastparquet')
data2.head()
data2.groupby('label').describe()
data2['verdict']=data2['label'].apply(lambda x: 1 if x=='spam' else 0)
data2.head()

,text,label,verdict
0,"Deezer.com 10,406,168 Artist DB\n\nWe have sc...",not_spam,0
1,🚨 ATTENTION ALL USERS! 🚨\n\n🆘 Are you looking ...,spam,1
2,I'm working on a stats project to test some of...,not_spam,0
3,"[[Sorry, I cannot generate inappropriate or sp...",spam,1
4,L@@k at these Unbelievable diet pills that can...,spam,1


In [39]:
X_train, X_test, y_train, y_test = train_test_split(data2.text,data2.verdict)
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:2]
model = MultinomialNB()
model.fit(X_train_count,y_train)
emails_count = v.transform(emails)
X_test_count = v.transform(X_test)
model.predict(emails_count), model.score(X_test_count, y_test)

(array([1, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int64), 0.9956011730205279)

In [41]:
from sentence_transformers import SentenceTransformer
import joblib

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load the saved classifier model
classifier = joblib.load('spam_classifier_model.pkl')

# Generate embeddings for the new emails
emails = [
    'Frii diskount here, bi fast',
    'click here to receive a free reward',
    'your car warranty is expired',
    'Do not fall for this trick!',
    'this is how much Dave saved with this simple trick',
    'you will not believe how simple this is!',
    'get your once a year discount today!',
    'receive your 30% discount here from your next product',
    'get your once a year discount today! Do not miss out on this HUGE oppurtunity!'
]
email_embeddings = model.encode(emails)

# Predict the labels for the new emails
email_predictions = classifier.predict(email_embeddings)

# Print the predictions
for email, prediction in zip(emails, email_predictions):
    label = 'Spam' if prediction == 1 else 'Non-Spam'
    print(f"Email: '{email}' is classified as {label}")
       

Email: 'Frii diskount here, bi fast' is classified as Non-Spam
Email: 'click here to receive a free reward' is classified as Spam
Email: 'your car warranty is expired' is classified as Non-Spam
Email: 'Do not fall for this trick!' is classified as Spam
Email: 'this is how much Dave saved with this simple trick' is classified as Spam
Email: 'you will not believe how simple this is!' is classified as Spam
Email: 'get your once a year discount today!' is classified as Spam
Email: 'receive your 30% discount here from your next product' is classified as Spam
Email: 'get your once a year discount today! Do not miss out on this HUGE oppurtunity!' is classified as Spam
